主要是为了对getProvince得到的数据，进行特征分析， 并与getCity的进行对比，找到这两种获取的数据集在xgboost上差别如此大的原因。尤其对应Part3部分。

分析点：

- 看看获取的数据集的大小的区别。


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from Pic.maxent_style import *
import datetime

from imblearn.over_sampling import SMOTE
from Pic.maxent_style import maxent_style,remove_palette
from sklearn.preprocessing import StandardScaler
from Pic.maxent_font import tick_font
from Utils.common.custerReadFile import custom_open
import re
from sklearn.preprocessing import MinMaxScaler
from Utils.common.custerReadFile import read_files
from Utils.dfp_new.get_data import readData

In [2]:
dir_ = "/Users/chaoxu/code/local-spark/Data/dfp_new"

In [3]:
df_c = readData(path=dir_)

In [4]:
df_c[['slope']] = df_c[['slope']].astype(float)
df_c[['label']] = df_c[['label']].fillna(0)

In [5]:
obj_df = df_c.select_dtypes(include=['object'])
for col in obj_df:
    dummy_col = pd.get_dummies(df_c[col])
    df_c = pd.concat([df_c, dummy_col], axis=1)
df_c.drop(obj_df, inplace=True, axis=1)

In [6]:
df_c['score'].describe()

count    8368.000000
mean        0.351755
std         0.474217
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: score, dtype: float64

In [ ]:
df_c.label.value_counts()

In [ ]:
df_c_t['score'].describe()

In [ ]:
df_c_t.label.value_counts()

In [ ]:
df_c.columns[df_c.isnull().any()]

In [ ]:
df_c_t.columns[df_c_t.isnull().any()]

In [ ]:
df_c_t['resolution'].head()

In [ ]:
df_c_t.head(10)

In [ ]:
df_c_t.resolution.describe()

In [ ]:
df_c.resolution.describe()

In [ ]:
df_c.loc[df_c.city_entropy.isnull()].shape

In [ ]:
df_c.loc[df_c.city_entropy.notnull()].shape

In [ ]:
df_c.city_entropy.shape

In [ ]:
df_c.loc[df_c.province_entropy.notnull()].shape

In [ ]:
df_c.loc[df_c.province_entropy.isnull()].shape

In [ ]:
df_c.province_entropy.shape

In [ ]:
df_c.loc[(df_c.province_entropy.isnull()) & (df_c.city_entropy.isnull())].shape

In [ ]:
df_c.loc[(df_c.province_entropy.notnull()) & (df_c.city_entropy.notnull())].shape

In [ ]:
df_c_t.loc[df_c_t.city_entropy.isnull()].shape

In [ ]:
df_c_t.loc[df_c_t.city_entropy.notnull()].shape

In [ ]:
df_c_t.city_entropy.shape

In [ ]:
df_c_t.loc[df_c_t.province_entropy.isnull()].shape

In [ ]:
df_c_t.loc[df_c_t.province_entropy.notnull()].shape

In [ ]:
df_c_t.loc[df_c_t.province_entropy.isnull()].shape

In [ ]:
df_c_t.province_entropy.shape

In [ ]:
df_c_t.loc[(df_c_t.province_entropy.isnull()) & (df_c_t.city_entropy.isnull())].shape

In [ ]:
df_c_t.loc[(df_c_t.province_entropy.notnull()) & (df_c_t.city_entropy.notnull())].shape

In [ ]:
df_c = df_c.fillna(-6.666)

In [ ]:
df_c_t = df_c_t.fillna(-6.666)

In [ ]:
cols = df_c.columns.difference(['label'])

In [ ]:
X = df_c[cols]
y = df_c.ix[:, df_c.columns == 'label']
v_features = cols
pic_nums = v_features.shape[0]
import matplotlib.gridspec as gridspec
plt.figure(figsize=(12,28*4))
gs = gridspec.GridSpec(pic_nums, 1)
for i, cn in enumerate(df_c[v_features]):
    print("cn",cn)
    ax = plt.subplot(gs[i])
    sns.distplot(df_c[cn][df_c.label == 1], bins=10)
    sns.distplot(df_c[cn][df_c.label == 0], bins=10)
    ax.set_xlabel('')
    ax.set_title('histogram of feature: ' + str(cn))
plt.show()

In [ ]:
drop_cols = ['baseEntropy','device_browser_engine_entropy',
              'device_model','jsid_entropy','resolution']

In [ ]:
df_c = df_c.drop(drop_cols,axis=1)

In [ ]:
df_c.columns

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
ts_match = re.compile('^.*ts_diff$')
ts_amatch = np.vectorize(lambda x:bool(ts_match.match(x)))
ts_cols = df_c.columns.values[ts_amatch(df_c.columns.values)]
ts_df_c = df_c[ts_cols]
ts_no = df_c[df_c.columns.difference(ts_cols)]

min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(ts_df_c)
ts_df_c = pd.DataFrame(X_scaled, columns = ts_df_c.columns)
df_c = pd.concat([ts_no, ts_df_c], axis=1)

In [ ]:
def tick_font_new(ax=None,font_size=None,rotation=90):
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(font_size)
        tick.label.set_rotation(rotation)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(font_size)
        tick.label.set_rotation(0)
    return None

In [ ]:
fig,ax= plt.subplots(figsize=(60,60))
sns.heatmap(df_c.corr(),ax=ax,annot=True,fmt='.1f',annot_kws={"size": 32})
tick_font(ax=ax,font_size=32,rotation_y=0)
plt.show()

In [ ]:
df_c = df_c.drop(['ts_diff'],axis=1)

In [ ]:
df_c.columns

In [ ]:
df_c.shape

In [ ]:
df_c_t.shape

In [ ]:
X = df_c_t[cols]
y = df_c_t.ix[:, df_c_t.columns == 'label']
v_features = cols
pic_nums = v_features.shape[0]
import matplotlib.gridspec as gridspec
plt.figure(figsize=(12,28*4))
gs = gridspec.GridSpec(pic_nums, 1)
for i, cn in enumerate(df_c_t[v_features]):
    print("cn",i,cn)
    ax = plt.subplot(gs[i])
    sns.distplot(df_c_t[cn][df_c_t.label == 1], bins=10)
    sns.distplot(df_c_t[cn][df_c_t.label == 0], bins=10)
    ax.set_xlabel('')
    ax.set_title('histogram of feature: ' + str(cn))
plt.show()